In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [11]:
df = pd.read_csv("b_cancer.csv")
df.iloc[[12]]

,Recurrence,Age,Menopause,Tumor-size,Inv-nodes,Node-caps,Deg-malign,Breast,Breast-quad,Irradiat
12,no-recurrence-events,60-69,lt40,Oct-14,0-2,no,1,left,right_up,no


In [3]:
df.dtypes

no-recurrence-events    object
30-39                   object
premeno                 object
30-34                   object
0-2                     object
no                      object
3                        int64
left                    object
left_low                object
no.1                    object
dtype: object

In [5]:
df = df.drop(df[df['label'] == 'good'].index, axis=0)
df = df.drop(df[df['label'] == 'vgood'].index, axis=0)
df.shape

(1594, 7)

In [96]:
labels = []
for i in df["label"]:
    if i == 'unacc':
        labels.append(0)
    else: labels.append(1)


labels = pd.DataFrame(labels)

In [7]:
features = df.drop(["label"], axis=1)
features

,buying,maint,doors,persons,lug-boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med
...,...,...,...,...,...,...
1716,low,low,5more,4,big,low
1719,low,low,5more,more,small,low
1720,low,low,5more,more,small,med
1722,low,low,5more,more,med,low


In [8]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=1)

train_features.reset_index(drop=True, inplace=True)
test_features.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
test_labels.reset_index(drop=True, inplace=True)

In [9]:
train = pd.concat([train_features, train_labels], axis=1)
train

,buying,maint,doors,persons,lug-boot,safety,0
0,high,high,5more,4,small,high,1
1,low,low,4,4,big,low,0
2,low,vhigh,4,more,med,low,0
3,low,low,5more,4,small,med,1
4,med,vhigh,5more,4,small,med,0
...,...,...,...,...,...,...,...
1110,high,med,4,4,med,med,1
1111,med,vhigh,3,4,med,high,1
1112,med,med,2,4,big,med,1
1113,vhigh,med,2,more,small,med,0


In [72]:
n_train = train.shape[0]
n_train

1115

In [158]:
class Naive_Bayes():
    def __init__(self):
        self.result =[]
        
    def priors(self, train):
        newdf = train[0].value_counts()
        pr=[]
        pr.append(newdf[0]/train.shape[0])
        pr.append(newdf[1]/train.shape[0])
        return pr
    
    def prob_data(self, train, y):
        post=1
        for key, values in y.iteritems():
            newy = y.loc[:, [key]]
            common_cols = list(set(train.columns) & set(newy.columns))
            newdf = pd.merge(train, newy, on=common_cols, how='inner')
            p = (newdf.shape[0])/train.shape[0]
            post *= p
        return post
    
    def prob_0(self, train, y):
        post_0=1
        for key, values in y.iteritems():
            newy = y.loc[:, [key]]
            common_cols = list(set(train.columns) & set(newy.columns))
            newdf = pd.merge(train, newy, on=common_cols, how='inner')
            count = newdf[(newdf[0]==0)]
            p=count.shape[0]/train.shape[0]
            post_0 *= p
        return post_0
    
    def prob_1(self, train, y):
        post_1=1
        for key, values in y.iteritems():
            newy = y.loc[:, [key]]
            common_cols = list(set(train.columns) & set(newy.columns))
            newdf = pd.merge(train, newy, on=common_cols, how='inner')
            count = newdf[(newdf[0]==1)]
            p=count.shape[0]/train.shape[0]
            post_1 *= p
        return post_1
    
    def bayes(self, prior, post, post_0, post_1):
        bayes0 = pr0*post_0/post
        bayes1 = pr1*post_1/post
        if (bayes0>bayes1):
            return 0
        else: return 1
        
    def predict(self, train, test_features):
        for i in range(test_features.shape[0]):
            y = test_features.iloc[[i]]
            y.reset_index(drop=True, inplace=True)
            
            prior = self.priors(train)
            post = self.prob_data(train, y)
            post_0 = self.prob_0(train, y)
            post_1 = self.prob_1(train, y)
            
            pred = self.bayes(prior, post, post_0, post_1)
            self.result.append(pred)
        

In [159]:
naive_bayes = Naive_Bayes()

In [160]:
naive_bayes.predict(train, test_features)

In [161]:
predict_label = pd.DataFrame(naive_bayes.result)
predict_label

,0
0,0
1,0
2,0
3,0
4,0
...,...
474,0
475,0
476,0
477,0


In [162]:
predict_label[0].value_counts()

0    479
Name: 0, dtype: int64

In [163]:
acc = metrics.accuracy_score(test_labels, predict_label)
acc

0.7724425887265136

In [164]:
prec = metrics.precision_score(test_labels, predict_label)
prec

c:\users\rochi\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0